# LSTM 

## Outline
1. [Imports](#imports)
2. [Data](#data)
3. [Model](#model)
5. [Train](#train)
6. [Predict](#predict)

<a name=imports></a>
## Imports

In [1]:
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

<a name=data></a>
## Data

### Ingestion 

In [2]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
print("Parent Directory", parentDir)
# Save the path to all of the datasets
india_cases_path = os.path.join(parentDir, "../cleaned_datasets/india/daily_cases_india.csv")
india_vacc_path = os.path.join(parentDir, "../cleaned_datasets/india/daily_vacc_india.csv")
usa_cases_path = os.path.join(parentDir, "../cleaned_datasets/usa/daily_cases_usa.csv")
usa_vacc_path = os.path.join(parentDir, "../cleaned_datasets/usa/vacc_usa.csv")

# Quick check to make sure the path exists
print("Path:", india_cases_path)
print("Exists:", os.path.exists(india_cases_path))

Parent Directory /covid19-prediction/models
Path: /covid19-prediction/models/../cleaned_datasets/india/daily_cases_india.csv
Exists: True


In [3]:
# Load the data as a pandas dataframe
india_cases_df = pd.read_csv(india_cases_path)
india_vacc_df =  pd.read_csv(india_vacc_path)

usa_cases_df = pd.read_csv(usa_cases_path)
usa_vacc_df = pd.read_csv(usa_vacc_path)

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('India Vacc:\n',india_vacc_df.head(),'\n')

print('USA Cases:\n',usa_cases_df.head(),'\n')
print('USA Vacc:\n',usa_vacc_df.head(),'\n')

India Cases:
    Unnamed: 0        Date  Confirmed  Deaths  Recovered  Active
0           0  2020-01-30        1.0     0.0        0.0     0.0
1           1  2020-01-31        0.0     0.0        0.0     0.0
2           2  2020-02-01        0.0     0.0        0.0     0.0
3           3  2020-02-02        1.0     0.0        0.0     0.0
4           4  2020-02-03        1.0     0.0        0.0     0.0 

India Vacc:
    Updated On  Total_Doses  First_Dose  Second_Dose
0  2021-01-16          NaN         NaN          NaN
1  2021-01-17      20656.0     20656.0          0.0
2  2021-01-18      81690.0     81690.0          0.0
3  2021-01-19     192152.0    192152.0          0.0
4  2021-01-20     111510.0    111510.0          0.0 

USA Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-04-12        NaN     NaN        NaN
1  2020-04-13    25322.0  1546.0    11785.0
2  2020-04-14    26713.0  2305.0     6484.0
3  2020-04-15    29380.0  2478.0     6093.0
4  2020-04-16    31542.0  4616.0     5234

### Pre-Processing

In [4]:
# Select only the Confirmed column for univariate analysis
# Selecting from the first index because the 0th index is NaN
india_cases_df = india_cases_df[["Confirmed"]][1:]
usa_cases_df = usa_cases_df[["Confirmed"]][1:]

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1        0.0
2        0.0
3        1.0
4        1.0
5        0.0 

USA Cases:
    Confirmed
1    25322.0
2    26713.0
3    29380.0
4    31542.0
5    32022.0 



In [5]:
# Normalize the data
india_cases_mean = india_cases_df.mean()
india_cases_std = india_cases_df.std()

usa_cases_mean = usa_cases_df.mean()
usa_cases_std = usa_cases_df.std()


india_cases_df = (india_cases_df-india_cases_mean)/india_cases_std
usa_cases_df = (usa_cases_df-usa_cases_mean)/usa_cases_std

# Visualize the datasets
print('India Cases:\n',india_cases_df.head(),'\n')
print('USA Cases:\n',usa_cases_df.head(),'\n')

India Cases:
    Confirmed
1  -0.687995
2  -0.687995
3  -0.687983
4  -0.687983
5  -0.687995 

USA Cases:
    Confirmed
1  -0.817861
2  -0.797170
3  -0.757499
4  -0.725340
5  -0.718200 



In [6]:
# Create train test splits
india_cases_train, india_cases_test = train_test_split(india_cases_df, test_size=0.2, shuffle=False)
india_vacc_train, india_vacc_test = train_test_split(india_vacc_df, test_size=0.2, shuffle=False)

usa_cases_train, usa_cases_test = train_test_split(usa_cases_df, test_size=0.2, shuffle=False)
usa_vacc_train, usa_vacc_test = train_test_split(usa_vacc_df, test_size=0.2, shuffle=False)

# Visualize splits
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
      Confirmed
1    -0.687995
2    -0.687995
3    -0.687983
4    -0.687983
5    -0.687995
..         ...
492   0.750239
493   0.576535
494   0.398886
495   0.475509
496   0.486403

[496 rows x 1 columns] 

USA Cases:
      Confirmed
1    -0.817861
2    -0.797170
3    -0.757499
4    -0.725340
5    -0.718200
..         ...
434  -1.128237
435  -1.020678
436  -1.035523
437  -1.001267
438  -0.989828

[438 rows x 1 columns] 



In [7]:
# Convert to numpy array
india_cases_train, india_cases_test = india_cases_train.to_numpy().flatten(), india_cases_test.to_numpy()
usa_cases_train, usa_cases_test = usa_cases_train.to_numpy().flatten(), usa_cases_test.to_numpy()

# Visualize outputs
print('India Cases:\n',india_cases_train,'\n')
print('USA Cases:\n',usa_cases_train,'\n')

India Cases:
 [-6.87995117e-01 -6.87995117e-01 -6.87982552e-01 -6.87982552e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01
 -6.87995117e-01 -6.87995117e-01 -6.87995117e-01 -6.87969986e-01
 -6.87995117e-01 -6.87706113e-01 -6.87969986e-01 -6.87982552e-01
 -6.87957421e-01 -6.87932290e-01 -6.87944855e-01 -6.87831767e-01
 -6.87919725e-01 -6.87856898e-01 -6.87882028e-01 -6.87743809e-01
 -6.87856898e-01 -6.87919725e-01 -6.87706113e-01 -6.87819202e-01
 -6.87517632e-01 -6.87366848e-01 -6.86914494e-01 -6.87165802e-01
 -6.86700882e-01 -6.87530198e-01 -6.86474705e-01 -6.87115540e-01
 -6.85984655e-01 -6.86738578e-01 -6.87530198e-01 -6.85142775e-01
 -6.8616057

In [8]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [9]:
# choose a number of time steps
n_steps = 3
# split into samples
india_cases_train_X, india_cases_train_y = split_sequence(india_cases_train, n_steps)
india_cases_test_X, india_cases_test_y = split_sequence(india_cases_test, n_steps)


# summarize the data
for i in range(0,5):
    print(india_cases_train_X[i], india_cases_train_y[i])

[-0.68799512 -0.68799512 -0.68798255] -0.687982551549953
[-0.68799512 -0.68798255 -0.68798255] -0.6879951169346211
[-0.68798255 -0.68798255 -0.68799512] -0.6879951169346211
[-0.68798255 -0.68799512 -0.68799512] -0.6879951169346211
[-0.68799512 -0.68799512 -0.68799512] -0.6879951169346211


In [10]:
# Reshape the data into [samples, timesteps, features]
# univariate
n_features = 1

india_cases_train_X = india_cases_train_X.reshape((india_cases_train_X.shape[0], 
                                                   india_cases_train_X.shape[1], n_features))
india_cases_test_X = india_cases_test_X.reshape((india_cases_test_X.shape[0], 
                                                 india_cases_test_X.shape[1], n_features))

# Visualize outputs
print('India Cases:\n',india_cases_train_X,'\n')

India Cases:
 [[[-0.68799512]
  [-0.68799512]
  [-0.68798255]]

 [[-0.68799512]
  [-0.68798255]
  [-0.68798255]]

 [[-0.68798255]
  [-0.68798255]
  [-0.68799512]]

 ...

 [[ 0.82649813]
  [ 0.75023881]
  [ 0.57653493]]

 [[ 0.75023881]
  [ 0.57653493]
  [ 0.39888553]]

 [[ 0.57653493]
  [ 0.39888553]
  [ 0.47550924]]] 



<a name=model></a>
## Model 

### Univariate non-stacked

In [24]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanAbsolutePercentageError, MeanAbsoluteError
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [63]:
# Define the model
def build_univariate_non_stacked(optimizer="adam", lstm_nparams=100, n_steps=3, n_features=1):
    model_uni = Sequential()
    model_uni.add(LSTM(lstm_nparams, activation='relu', input_shape=(n_steps, n_features)))
    model_uni.add(Dense(1))
    model_uni.compile(optimizer=optimizer,loss='mae', metrics=["accuracy"])
    model_uni.summary()
    return model_uni

#### Grid Search

In [59]:
# Using KerasClassifier wrapper 
keras_estimator = KerasClassifier(build_fn=build_univariate_non_stacked, verbose=1)

In [60]:
# Defining parameters for the gridserach
param_grid = {
    'epochs': [10,100,300,],
    'lstm_nparams':[15,50,100,150],
    'n_steps': [3, 6, 15],
#     'optimizer': ['RMSprop','Adam','Adamax','sgd']
}

kfold_splits = 5

In [61]:
# Defining GridSearch
grid = GridSearchCV(estimator=keras_estimator,
                   verbose=-1,
                   return_train_score=True,
                   cv=kfold_splits,
                   param_grid=param_grid,
)

In [62]:
# Fitting GridSearch
grid_result = grid.fit(india_cases_train_X, india_cases_train_y, )

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Model: "sequential_165"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_166 (LSTM)              (None, 15)                1020      
_________________________________________________________________
dense_165 (Dense)            (None, 1)                 16        
Total params: 1,036
Trainable params: 1,036
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
13/13 [==============================] - 1s 5ms/step - loss: 192.8846 - accuracy: 0.0025
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 192.8475 - accuracy: 0.0025
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 192.8089 - accuracy: 0.0025
Epoch 4/10
13/13 [==============================] - 0s 4ms/step - loss: 192.7649 - accuracy: 0.0025
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 192.7120 - accuracy: 0.0025
Epoch 6/10
1

13/13 [==============================] - 1s 5ms/step - loss: 161.6573 - accuracy: 0.0025
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 161.5805 - accuracy: 0.0025
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 161.4916 - accuracy: 0.0025
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 161.3816 - accuracy: 0.0658
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 161.2342 - accuracy: 0.0785
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 161.0392 - accuracy: 0.0759
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 160.8157 - accuracy: 0.0759
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 160.5010 - accuracy: 0.0759
Epoch 9/10
13/13 [==============================] - 0s 4ms/step - loss: 160.0178 - accuracy: 0.0759
Epoch 10/10
13/13 [==============================] - 0s 2ms/step - loss: 158.6616 - accuracy: 0.0785
Model: "se

13/13 [==============================] - 1s 4ms/step - loss: 162.1643 - accuracy: 0.0025
Epoch 2/10
13/13 [==============================] - 0s 4ms/step - loss: 162.1167 - accuracy: 0.0025
Epoch 3/10
13/13 [==============================] - 0s 4ms/step - loss: 162.0657 - accuracy: 0.0025
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 162.0087 - accuracy: 0.0025
Epoch 5/10
13/13 [==============================] - 0s 4ms/step - loss: 161.9381 - accuracy: 0.0025
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 161.8468 - accuracy: 0.0025
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 161.7206 - accuracy: 0.0025
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 161.5426 - accuracy: 0.0025
Epoch 9/10
13/13 [==============================] - 0s 5ms/step - loss: 161.2642 - accuracy: 0.0000e+00
Epoch 10/10
13/13 [==============================] - 0s 1ms/step - loss: 160.4583 - accuracy: 0.0761
Model:

13/13 [==============================] - 1s 5ms/step - loss: 193.0313 - accuracy: 0.0025
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 192.9704 - accuracy: 0.0025
Epoch 3/10
13/13 [==============================] - 0s 5ms/step - loss: 192.9167 - accuracy: 0.0025
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 192.8698 - accuracy: 0.0025
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 192.8245 - accuracy: 0.0025
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 192.7799 - accuracy: 0.0025
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 192.7336 - accuracy: 0.0025
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 192.6827 - accuracy: 0.0025
Epoch 9/10
13/13 [==============================] - 0s 5ms/step - loss: 192.6249 - accuracy: 0.0025
Epoch 10/10
13/13 [==============================] - 0s 1ms/step - loss: 192.5095 - accuracy: 0.0076
Model: "se

13/13 [==============================] - 1s 5ms/step - loss: 163.7306 - accuracy: 0.0759
Epoch 2/10
13/13 [==============================] - 0s 4ms/step - loss: 163.6840 - accuracy: 0.0759
Epoch 3/10
13/13 [==============================] - 0s 5ms/step - loss: 163.6382 - accuracy: 0.0759
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 163.5942 - accuracy: 0.0759
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 163.5448 - accuracy: 0.0759
Epoch 6/10
13/13 [==============================] - 0s 4ms/step - loss: 163.4805 - accuracy: 0.0759
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 163.4000 - accuracy: 0.0759
Epoch 8/10
13/13 [==============================] - 0s 4ms/step - loss: 163.2789 - accuracy: 0.0759
Epoch 9/10
13/13 [==============================] - 0s 5ms/step - loss: 163.1020 - accuracy: 0.0734
Epoch 10/10
13/13 [==============================] - 0s 2ms/step - loss: 162.5844 - accuracy: 0.0734
Model: "se

...

13/13 [==============================] - 0s 5ms/step - loss: 13.2897 - accuracy: 0.0051
Epoch 258/300
13/13 [==============================] - 0s 5ms/step - loss: 13.7194 - accuracy: 0.0051
Epoch 259/300
13/13 [==============================] - 0s 5ms/step - loss: 13.0452 - accuracy: 0.0051
Epoch 260/300
13/13 [==============================] - 0s 4ms/step - loss: 13.1547 - accuracy: 0.0051
Epoch 261/300
13/13 [==============================] - 0s 5ms/step - loss: 13.1114 - accuracy: 0.0076
Epoch 262/300
13/13 [==============================] - 0s 4ms/step - loss: 12.9570 - accuracy: 0.0152
Epoch 263/300
13/13 [==============================] - 0s 4ms/step - loss: 12.7239 - accuracy: 0.0127
Epoch 264/300
13/13 [==============================] - 0s 5ms/step - loss: 12.9870 - accuracy: 0.0076
Epoch 265/300
13/13 [==============================] - 0s 5ms/step - loss: 12.8485 - accuracy: 0.0101
Epoch 266/300
13/13 [==============================] - 0s 4ms/step - loss: 14.2916 - accuracy: 0

13/13 [==============================] - 0s 5ms/step - loss: 41.6886 - accuracy: 0.0785
Epoch 19/300
13/13 [==============================] - 0s 5ms/step - loss: 40.9474 - accuracy: 0.0785
Epoch 20/300
13/13 [==============================] - 0s 5ms/step - loss: 40.5932 - accuracy: 0.0785
Epoch 21/300
13/13 [==============================] - 0s 5ms/step - loss: 40.0079 - accuracy: 0.0785
Epoch 22/300
13/13 [==============================] - 0s 5ms/step - loss: 39.6923 - accuracy: 0.0785
Epoch 23/300
13/13 [==============================] - 0s 5ms/step - loss: 39.3526 - accuracy: 0.0785
Epoch 24/300
13/13 [==============================] - 0s 5ms/step - loss: 38.3512 - accuracy: 0.0785
Epoch 25/300
13/13 [==============================] - 0s 5ms/step - loss: 38.4173 - accuracy: 0.0785
Epoch 26/300
13/13 [==============================] - 0s 5ms/step - loss: 37.5365 - accuracy: 0.0785
Epoch 27/300
13/13 [==============================] - 0s 5ms/step - loss: 36.8444 - accuracy: 0.0785
Epo

13/13 [==============================] - 0s 4ms/step - loss: 20.9788 - accuracy: 0.0785
Epoch 100/300
13/13 [==============================] - 0s 5ms/step - loss: 19.9933 - accuracy: 0.0785
Epoch 101/300
13/13 [==============================] - 0s 5ms/step - loss: 20.3219 - accuracy: 0.0785
Epoch 102/300
13/13 [==============================] - 0s 4ms/step - loss: 20.2398 - accuracy: 0.0785
Epoch 103/300
13/13 [==============================] - 0s 4ms/step - loss: 20.1951 - accuracy: 0.0785
Epoch 104/300
13/13 [==============================] - 0s 5ms/step - loss: 20.1110 - accuracy: 0.0785
Epoch 105/300
13/13 [==============================] - 0s 5ms/step - loss: 20.2239 - accuracy: 0.0785
Epoch 106/300
13/13 [==============================] - 0s 5ms/step - loss: 20.1062 - accuracy: 0.0785
Epoch 107/300
13/13 [==============================] - 0s 5ms/step - loss: 20.2781 - accuracy: 0.0785
Epoch 108/300
13/13 [==============================] - 0s 5ms/step - loss: 19.7743 - accuracy: 0

13/13 [==============================] - 0s 5ms/step - loss: 19.3970 - accuracy: 0.0785
Epoch 180/300
13/13 [==============================] - 0s 5ms/step - loss: 19.5468 - accuracy: 0.0785
Epoch 181/300
13/13 [==============================] - 0s 5ms/step - loss: 19.1863 - accuracy: 0.0785
Epoch 182/300
13/13 [==============================] - 0s 5ms/step - loss: 19.3300 - accuracy: 0.0785
Epoch 183/300
13/13 [==============================] - 0s 4ms/step - loss: 19.5516 - accuracy: 0.0785
Epoch 184/300
13/13 [==============================] - 0s 4ms/step - loss: 19.5842 - accuracy: 0.0785
Epoch 185/300
13/13 [==============================] - 0s 4ms/step - loss: 19.9037 - accuracy: 0.0785
Epoch 186/300
13/13 [==============================] - 0s 5ms/step - loss: 19.3094 - accuracy: 0.0785
Epoch 187/300
13/13 [==============================] - 0s 5ms/step - loss: 19.1529 - accuracy: 0.0785
Epoch 188/300
13/13 [==============================] - 0s 5ms/step - loss: 19.5568 - accuracy: 0

13/13 [==============================] - 0s 5ms/step - loss: 18.2869 - accuracy: 0.0785
Epoch 260/300
13/13 [==============================] - 0s 5ms/step - loss: 18.5626 - accuracy: 0.0785
Epoch 261/300
13/13 [==============================] - 0s 5ms/step - loss: 18.5899 - accuracy: 0.0785
Epoch 262/300
13/13 [==============================] - 0s 5ms/step - loss: 18.4540 - accuracy: 0.0785
Epoch 263/300
13/13 [==============================] - 0s 5ms/step - loss: 18.5956 - accuracy: 0.0785
Epoch 264/300
13/13 [==============================] - 0s 5ms/step - loss: 18.2821 - accuracy: 0.0785
Epoch 265/300
13/13 [==============================] - 0s 5ms/step - loss: 18.6392 - accuracy: 0.0785
Epoch 266/300
13/13 [==============================] - 0s 4ms/step - loss: 18.5503 - accuracy: 0.0785
Epoch 267/300
13/13 [==============================] - 0s 5ms/step - loss: 18.0849 - accuracy: 0.0785
Epoch 268/300
13/13 [==============================] - 0s 5ms/step - loss: 18.3216 - accuracy: 0

In [90]:
# Writing the gridsearch results to file
file1 = open("univariate_non_stacked.txt", "w")
file1.write("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
file1.write("\n")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    file1.write("%f (%f) with: %r" % (mean, stdev, param))
    file1.write("\n")
file1.close()

In [66]:
sorted(grid_result.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_epochs',
 'param_lstm_nparams',
 'param_n_steps',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [66]:
sorted(grid_result.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_epochs',
 'param_lstm_nparams',
 'param_n_steps',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [88]:
grid_result.cv_results_['mean_train_score']

array([0.04868856, 0.03498169, 0.06235944, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.04868856, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.04868856, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.04868856, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.04868856, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.06185311, 0.04868856, 0.04868856,
       0.04868856, 0.04868856, 0.04868856, 0.04868856, 0.04868856,
       0.03447536])

### Univariate stacked 

In [17]:
# Define the model
def build_univariate_non_stacked(lstm_nparams_l1, lstm_nparams_l2):
    model_uni_stacked = Sequential()
    model_uni_stacked.add(LSTM(lstm_nparams_l1, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model_uni_stacked.add(LSTM(lstm_nparams_l2, activation='relu'))
    model_uni_stacked.add(Dense(1))
    model_uni_stacked.compile(optimizer='adam',loss='mae')
    model_uni_stacked.summary()
    return model_uni_stacked

## Multivariate 

In [14]:
# Define the model
model_uni = Sequential()
model_uni.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model_uni.add(Dense(1))
model_uni.compile(optimizer='adam',loss='mae')
model_uni.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


<a name=train></a>
## Train 

In [64]:
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Univariate non-stacked

In [65]:
# Fit the model
model_uni = build_univariate_non_stacked()
model_uni.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

model_uni.save('univar_1_50.h5')

Model: "sequential_346"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_347 (LSTM)              (None, 100)               40800     
_________________________________________________________________
dense_346 (Dense)            (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
16/16 [==============================] - 1s 27ms/step - loss: 0.6213 - accuracy: 0.0000e+00 - val_loss: 0.1394 - val_accuracy: 0.0000e+00
Epoch 2/500
16/16 [==============================] - 0s 6ms/step - loss: 0.4746 - accuracy: 0.0000e+00 - val_loss: 0.0729 - val_accuracy: 0.0000e+00
Epoch 3/500
16/16 [==============================] - 0s 6ms/step - loss: 0.2761 - accuracy: 0.0000e+00 - val_loss: 0.0702 - val_accuracy: 0.0000e+00
Epoch 4/500
16/16 [==============================] - 0s 6ms/st

16/16 [==============================] - 0s 6ms/step - loss: 0.0699 - accuracy: 0.0000e+00 - val_loss: 0.0663 - val_accuracy: 0.0000e+00
Epoch 51/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0764 - accuracy: 0.0000e+00 - val_loss: 0.0645 - val_accuracy: 0.0000e+00
Epoch 52/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0689 - accuracy: 0.0000e+00 - val_loss: 0.0711 - val_accuracy: 0.0000e+00
Epoch 53/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0733 - accuracy: 0.0000e+00 - val_loss: 0.0688 - val_accuracy: 0.0000e+00
Epoch 54/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0690 - accuracy: 0.0000e+00 - val_loss: 0.0707 - val_accuracy: 0.0000e+00
Epoch 55/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0701 - accuracy: 0.0000e+00 - val_loss: 0.0643 - val_accuracy: 0.0000e+00
Epoch 56/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0677 - accuracy: 0.0000e+00 - val_los

Epoch 105/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0664 - accuracy: 0.0000e+00 - val_loss: 0.0671 - val_accuracy: 0.0000e+00
Epoch 106/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0670 - accuracy: 0.0000e+00 - val_loss: 0.0673 - val_accuracy: 0.0000e+00
Epoch 107/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0672 - accuracy: 0.0000e+00 - val_loss: 0.0642 - val_accuracy: 0.0000e+00
Epoch 108/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0670 - accuracy: 0.0000e+00 - val_loss: 0.0647 - val_accuracy: 0.0000e+00
Epoch 109/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0657 - accuracy: 0.0000e+00 - val_loss: 0.0663 - val_accuracy: 0.0000e+00
Epoch 110/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0656 - accuracy: 0.0000e+00 - val_loss: 0.0684 - val_accuracy: 0.0000e+00
Epoch 111/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0646 - accuracy: 

16/16 [==============================] - 0s 6ms/step - loss: 0.0676 - accuracy: 0.0000e+00 - val_loss: 0.0646 - val_accuracy: 0.0000e+00
Epoch 160/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0667 - accuracy: 0.0000e+00 - val_loss: 0.0653 - val_accuracy: 0.0000e+00
Epoch 161/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0677 - accuracy: 0.0000e+00 - val_loss: 0.0679 - val_accuracy: 0.0000e+00
Epoch 162/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0643 - accuracy: 0.0000e+00 - val_loss: 0.0655 - val_accuracy: 0.0000e+00
Epoch 163/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.0000e+00 - val_loss: 0.0660 - val_accuracy: 0.0000e+00
Epoch 164/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0645 - accuracy: 0.0000e+00 - val_loss: 0.0669 - val_accuracy: 0.0000e+00
Epoch 165/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0656 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.0000e+00 - val_loss: 0.0660 - val_accuracy: 0.0000e+00
Epoch 214/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0628 - accuracy: 0.0000e+00 - val_loss: 0.0664 - val_accuracy: 0.0000e+00
Epoch 215/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0626 - accuracy: 0.0000e+00 - val_loss: 0.0636 - val_accuracy: 0.0000e+00
Epoch 216/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0617 - accuracy: 0.0000e+00 - val_loss: 0.0646 - val_accuracy: 0.0000e+00
Epoch 217/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0623 - accuracy: 0.0000e+00 - val_loss: 0.0625 - val_accuracy: 0.0000e+00
Epoch 218/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0618 - accuracy: 0.0000e+00 - val_loss: 0.0649 - val_accuracy: 0.0000e+00
Epoch 219/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0614 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0603 - accuracy: 0.0000e+00 - val_loss: 0.0581 - val_accuracy: 0.0000e+00
Epoch 268/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0609 - accuracy: 0.0000e+00 - val_loss: 0.0615 - val_accuracy: 0.0000e+00
Epoch 269/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0587 - accuracy: 0.0000e+00 - val_loss: 0.0604 - val_accuracy: 0.0000e+00
Epoch 270/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0581 - accuracy: 0.0000e+00 - val_loss: 0.0587 - val_accuracy: 0.0000e+00
Epoch 271/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0580 - accuracy: 0.0000e+00 - val_loss: 0.0609 - val_accuracy: 0.0000e+00
Epoch 272/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0597 - accuracy: 0.0000e+00 - val_loss: 0.0625 - val_accuracy: 0.0000e+00
Epoch 273/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0607 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0567 - accuracy: 0.0000e+00 - val_loss: 0.0564 - val_accuracy: 0.0000e+00
Epoch 322/500
16/16 [==============================] - 0s 5ms/step - loss: 0.0557 - accuracy: 0.0000e+00 - val_loss: 0.0635 - val_accuracy: 0.0000e+00
Epoch 323/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0590 - accuracy: 0.0000e+00 - val_loss: 0.0598 - val_accuracy: 0.0000e+00
Epoch 324/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0559 - accuracy: 0.0000e+00 - val_loss: 0.0545 - val_accuracy: 0.0000e+00
Epoch 325/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0578 - accuracy: 0.0000e+00 - val_loss: 0.0564 - val_accuracy: 0.0000e+00
Epoch 326/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0569 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 327/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0572 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0537 - accuracy: 0.0000e+00 - val_loss: 0.0552 - val_accuracy: 0.0000e+00
Epoch 376/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0530 - accuracy: 0.0000e+00 - val_loss: 0.0579 - val_accuracy: 0.0000e+00
Epoch 377/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0541 - accuracy: 0.0000e+00 - val_loss: 0.0562 - val_accuracy: 0.0000e+00
Epoch 378/500
16/16 [==============================] - 0s 5ms/step - loss: 0.0542 - accuracy: 0.0000e+00 - val_loss: 0.0526 - val_accuracy: 0.0000e+00
Epoch 379/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0565 - accuracy: 0.0000e+00 - val_loss: 0.0572 - val_accuracy: 0.0000e+00
Epoch 380/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0565 - accuracy: 0.0000e+00 - val_loss: 0.0558 - val_accuracy: 0.0000e+00
Epoch 381/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0583 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0532 - accuracy: 0.0000e+00 - val_loss: 0.0505 - val_accuracy: 0.0000e+00
Epoch 430/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0534 - accuracy: 0.0000e+00 - val_loss: 0.0508 - val_accuracy: 0.0000e+00
Epoch 431/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0545 - accuracy: 0.0000e+00 - val_loss: 0.0511 - val_accuracy: 0.0000e+00
Epoch 432/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0535 - accuracy: 0.0000e+00 - val_loss: 0.0580 - val_accuracy: 0.0000e+00
Epoch 433/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0541 - accuracy: 0.0000e+00 - val_loss: 0.0505 - val_accuracy: 0.0000e+00
Epoch 434/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0518 - accuracy: 0.0000e+00 - val_loss: 0.0558 - val_accuracy: 0.0000e+00
Epoch 435/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0521 - accuracy: 0.0000e+00 - v

16/16 [==============================] - 0s 6ms/step - loss: 0.0508 - accuracy: 0.0000e+00 - val_loss: 0.0480 - val_accuracy: 0.0000e+00
Epoch 484/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0522 - accuracy: 0.0000e+00 - val_loss: 0.0496 - val_accuracy: 0.0000e+00
Epoch 485/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0519 - accuracy: 0.0000e+00 - val_loss: 0.0516 - val_accuracy: 0.0000e+00
Epoch 486/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0518 - accuracy: 0.0000e+00 - val_loss: 0.0487 - val_accuracy: 0.0000e+00
Epoch 487/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0512 - accuracy: 0.0000e+00 - val_loss: 0.0580 - val_accuracy: 0.0000e+00
Epoch 488/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.0000e+00 - val_loss: 0.0503 - val_accuracy: 0.0000e+00
Epoch 489/500
16/16 [==============================] - 0s 6ms/step - loss: 0.0517 - accuracy: 0.0000e+00 - v

### Univariate stacked 

In [ ]:
# Fit the model
model_uni_stacked.fit(india_cases_train_X,
          india_cases_train_y, 
          validation_data=(india_cases_test_X, india_cases_test_y),
          epochs=500,
          callbacks=[tensorboard_callback])

model_uni_stacked.save('univar_2_50.h5')

<a name=predict></a>
## Predict

In [ ]:
india_cases_test_X

### Univariate 

In [ ]:
x_input = np.array(india_cases_test_X)
print(x_input)
yhat = model_uni.predict(x_input)
print(yhat)

### Univariate stacked 

### Univariate 

In [ ]:
x_input = np.array(india_cases_test_X)
print(x_input)
yhat_stacked = model_uni_stacked.predict(x_input)
print(yhat)

## Evaluate

### Univariate 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(india_cases_test_y, yhat).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(india_cases_test_y, yhat).numpy()

### Univariate stacked 

In [ ]:
mape(india_cases_test_y, yhat_stacked).numpy()